In [21]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sys
import matplotlib.pyplot as plt

L1_SIZE        = 10            # How many nodes in layer 1
EPOCHS         = 500
TEST_PCT       = 0.15
FEATURE_COUNT  = 2
LEARNING_RATE  = .01
SUMMARIES_DIR  = '/home/tom/tf_logs' # where to store Tensorboard data
LAMBDA         = 0.              # Regularization parameter
REGULARIZATION = 'L2'
#REGULARIZATION = 'DROPOUT'
PLOT           = False

In [22]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, dtype=tf.float32))

#### Get the data

In [23]:
if sys.platform[:3] =='win':
    data_loc = 'D:/Data/'
else:
    data_loc = "/home/tom/data/"

filenm = "haganCaseStudy1.csv"
df = pd.read_csv(data_loc+filenm, delimiter = ',')

# v1 and v2 is the voltage coming out of the 2 solar cells
# y is the position of the ping pong ball (in front of a light)
df.head()

,v1,v2,y
0,2.66,4.44,0.00
1,1.98,4.43,0.06
2,1.44,4.43,0.11
3,0.71,4.43,0.18
4,0.46,4.41,0.20


In [24]:
# Standardize the data
v1     = df['v1'].values.reshape(-1,1)
v1_std = StandardScaler().fit_transform(v1)
v2     = df['v2'].values.reshape(-1,1)
v2_std = StandardScaler().fit_transform(v2)
y      = df['y'].values.reshape(-1,1)
y_std  = StandardScaler().fit_transform(y)

In [25]:
# Split into train and test
# Recombine into DF
arr = np.concatenate([v1_std,v2_std,y_std], axis=1)
cols = ['v1','v2','y']
df = pd.DataFrame(arr, columns=cols)

train, test = train_test_split(df, test_size=TEST_PCT)

In [26]:
X = train[['v1', 'v2']]
Y = train[['y']]
X_test = test[['v1', 'v2']]
Y_test = test[['y']]

In [27]:
# See what the data looks like
if PLOT:
    a = plt.scatter(y_std, v1_std, s=5, color='blue',label='Voltage1')
    b = plt.scatter(y_std, v2_std, s=5, color='red', label='Voltage2')
    plt.legend(loc='lower right')
    plt.xlabel('Position', fontsize=12)
    plt.ylabel('Voltages', fontsize=12)
    plt.show()

In [28]:
# Set up the network
tf.reset_default_graph()
x  = tf.placeholder("float", shape=[None, FEATURE_COUNT])
y_ = tf.placeholder("float", shape=[None, 1])

l1_w = init_weights([FEATURE_COUNT, L1_SIZE])
l1_b = init_weights([1, L1_SIZE])
l1_act = tf.nn.tanh(tf.matmul(x,l1_w + l1_b))

l2_w = init_weights([L1_SIZE, 1])
l2_b = init_weights([1,1])

l2_out = tf.matmul(l1_act, l2_w + l2_b)

if REGULARIZATION == 'L2':
    print("regularizing")
    cost = tf.div(tf.add(
            tf.reduce_sum(tf.square(l2_out - y_)),
            tf.mul(LAMBDA, tf.reduce_sum(tf.square(l1_w)))
        ), 2*X.size)
else:
    cost = tf.reduce_sum(tf.square(l2_out - y_))

# Optimizer
optimize = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(cost)

# Accuracy
score = tf.reduce_sum(tf.square(l2_out - y_))
tf.scalar_summary('score', score)

merged = tf.merge_all_summaries()

regularizing


In [29]:
count = 1
with tf.Session() as sess:
    train_writer = tf.train.SummaryWriter(SUMMARIES_DIR + '/L2d', sess.graph)
    for LAMBDA in np.linspace(.1,.3,20):
        sess.run(tf.initialize_all_variables())
        for i in range(EPOCHS):
            sess.run(optimize, feed_dict = {x: X, y_: Y})
        s, m = sess.run([score, merged], feed_dict = {x: X_test, y_: Y_test})
        train_writer.add_summary(m, count)
        count += 1
    #tom = sess.run(l2_out, feed_dict = {x: X_test, y_: Y_test})
    train_writer.close()

In [30]:
# See what the data looks like
#plt.plot(X_test['v1'], tom, 'xb-')
if PLOT:
    a = plt.plot(X_test['v1'], tom,  color='blue',label='Prediction')
    b = plt.plot(X_test['v1'],Y_test,  color='red',label='Actual')
    plt.legend(loc='lower right')
    plt.xlabel('Position', fontsize=12)
    plt.ylabel('Voltages', fontsize=12)
    plt.show()